In [1]:
!pip install oracledb

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import time
from tqdm import tqdm

# API 설정
list_api_url = "http://kopis.or.kr/openApi/restful/pblprfr"
detail_api_url = "http://kopis.or.kr/openApi/restful/pblprfr/"
api_key = "4a97d2970cd44a27a8aa49b31f7d2fb0"

# 목록 및 상세 데이터 동시 수집
data = []
page = 1
rows = 10
request_interval = 1  # 요청 간격 (초)

print("API에서 데이터 수집 중...")
while True:
    # 목록 API 호출
    list_params = {
        "service": api_key,
        "stdate": "20250101",
        "eddate": "20250131",
        "cpage": page,
        "rows": rows,
        "shcate": "GGGA"
    }
    list_response = requests.get(list_api_url, params=list_params)
    if list_response.status_code == 200:
        root = ET.fromstring(list_response.text)
        page_data = [
            {
                "mt20id": item.findtext("mt20id"),
                "prfnm": item.findtext("prfnm"),
                "poster": item.findtext("poster"),
                "prfpdfrom": item.findtext("prfpdfrom"),
                "prfpdto": item.findtext("prfpdto")
            }
            for item in root.findall("db")
        ]
        if not page_data:  # 페이지에 데이터가 없으면 종료
            break

        # 상세 API 호출 및 데이터 수집
        for item in tqdm(page_data, desc=f"페이지 {page} 상세 데이터 수집 진행"):
            mt20id = item["mt20id"]
            detail_url = f"{detail_api_url}{mt20id}?service={api_key}"
            detail_response = requests.get(detail_url)
            if detail_response.status_code == 200:
                detail_root = ET.fromstring(detail_response.text)

                # 출연진 정보 확인
                prfcast = detail_root.findtext(".//prfcast", default="N/A")
                if prfcast and prfcast.strip() not in ["N/A", ""]:
                    styurls_element = detail_root.find(".//styurls")
                    if styurls_element is not None:
                        styurls = [styurl.text.strip() for styurl in styurls_element.findall("styurl") if styurl.text]
                        styurls_combined = "; ".join(styurls)
                    else:
                        styurls_combined = "없음"

                    # 데이터 저장
                    data.append({
                        "mt20id": mt20id,
                        "prfnm": item["prfnm"],
                        "poster": item["poster"],
                        "prfpdfrom": item["prfpdfrom"],
                        "prfpdto": item["prfpdto"],
                        "prfcast": prfcast,
                        "fcltynm": detail_root.findtext(".//fcltynm", default="N/A"),
                        "dtguidance": detail_root.findtext(".//dtguidance", default="N/A"),
                        "prfruntime": detail_root.findtext(".//prfruntime", default="N/A"),
                        "pcseguidance": detail_root.findtext(".//pcseguidance", default="N/A"),
                        "styurls": styurls_combined,
                        "musicallicense": detail_root.findtext(".//musicallicense", default="N/A"),
                        "musicalcreate": detail_root.findtext(".//musicalcreate", default="N/A")
                    })
            time.sleep(request_interval)
        page += 1
    else:
        break

# 데이터프레임 생성
final_df = pd.DataFrame(data)


API에서 데이터 수집 중...


페이지 21 상세 데이터 수집 진행:  10%|█████                                             | 1/10 [00:01<00:09,  1.10s/it]

In [3]:
import oracledb

# Oracle Instant Client 경로 설정 (Thick 모드 사용)
oracledb.init_oracle_client(lib_dir=r"C:/oraclexe/app/oracle/instantclient_23_6")

# Oracle DB 연결 설정
connection = oracledb.connect(
    user="hr",
    password="12345",
    dsn="localhost:1521/xe"
)
cursor = connection.cursor()

# 데이터베이스 삽입
print("데이터베이스에 데이터 삽입 중...")

# tb_musical 데이터 삽입 및 갱신
for _, row in final_df.iterrows():
    cursor.execute("""
        MERGE INTO tb_musical USING DUAL
        ON (musical_id = :musical_id)
        WHEN MATCHED THEN
        UPDATE SET
            musical_title = :musical_title,
            musical_poster = :musical_poster,
            musical_st_dt = TO_DATE(:musical_st_dt, 'YYYY.MM.DD'),
            musical_ed_dt = TO_DATE(:musical_ed_dt, 'YYYY.MM.DD'),
            musical_license = :musical_license,
            musical_create = :musical_create,
            musical_cast = :musical_cast
        WHEN NOT MATCHED THEN
        INSERT (musical_id, musical_title, musical_poster, musical_st_dt, musical_ed_dt, musical_license, musical_create, musical_cast)
        VALUES (:musical_id, :musical_title, :musical_poster, TO_DATE(:musical_st_dt, 'YYYY.MM.DD'), TO_DATE(:musical_ed_dt, 'YYYY.MM.DD'), :musical_license, :musical_create, :musical_cast)
    """, {
        "musical_id": row["mt20id"],
        "musical_title": row["prfnm"],
        "musical_poster": row["poster"],
        "musical_st_dt": row["prfpdfrom"],
        "musical_ed_dt": row["prfpdto"],
        "musical_license": row["musicallicense"],
        "musical_create": row["musicalcreate"],
        "musical_cast": row["prfcast"]
    })

# tb_show 데이터 삽입 및 갱신
for _, row in final_df.iterrows():
    cursor.execute("""
        MERGE INTO tb_show USING DUAL
        ON (musical_id = :musical_id AND hall_name = :hall_name AND show_dt = :show_dt)
        WHEN MATCHED THEN
        UPDATE SET
            show_runtime = :show_runtime,
            show_price = :show_price,
            show_cast = :show_cast,
            show_imgs = :show_imgs
        WHEN NOT MATCHED THEN
        INSERT (musical_id, hall_name, show_dt, show_runtime, show_price, created_at, show_cast, show_imgs)
        VALUES (:musical_id, :hall_name, :show_dt, :show_runtime, :show_price, CURRENT_TIMESTAMP, :show_cast, :show_imgs)
    """, {
        "musical_id": row["mt20id"],
        "hall_name": row["fcltynm"],
        "show_dt": row["dtguidance"],
        "show_runtime": row["prfruntime"],
        "show_price": row["pcseguidance"],
        "show_cast": row["prfcast"],
        "show_imgs": row["styurls"]
    })

connection.commit()
connection.close()

print("데이터베이스 삽입 완료!")

데이터베이스에 데이터 삽입 중...
데이터베이스 삽입 완료!


In [11]:
list_df.to_csv("list_data.csv", index=False, encoding="utf-8-sig")
detail_df.to_csv("detail_data.csv", index=False, encoding="utf-8-sig")

In [3]:
final_df.to_csv("final_df.csv", index=False, encoding="utf-8-sig")

mt20id: PF150471, prfcast: '박선령, 허혁, 조재문, 이혜림'
